In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, ReLU, LeakyReLU, GlobalAveragePooling2D, BatchNormalization, Dropout, RandomFlip, RandomRotation, RandomZoom, RandomContrast, Rescaling, RandomTranslation, RandomBrightness
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.models import load_model

2025-08-29 04:57:25.391458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756443445.639635      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756443445.714432      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
(x_train, y_train), (x_test,y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [3]:
x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0
batch_size = 32

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [4]:
model = Sequential([
    Input(shape=(32,32,3)),

    Conv2D(32, (3,3), padding="same", activation=None, kernel_initializer=HeNormal()),
    ReLU(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), padding="same", activation=None, kernel_initializer=HeNormal()),
    ReLU(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(96, (3,3), padding="same" , activation=None, kernel_initializer=HeNormal()),
    ReLU(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), padding="same" , activation=None, kernel_initializer=HeNormal()),
    ReLU(),
    MaxPooling2D(pool_size=(2,2)),

    GlobalAveragePooling2D(),

    Dense(128, activation=None, kernel_initializer=HeNormal()),
    ReLU(),
    Dropout(0.5),

    Dense(64, activation=None, kernel_initializer=HeNormal()),
    ReLU(),
    Dropout(0.5),

    Dense(10, activation='softmax') # Updated output dense layer to 10 for CIFAR-10
])

2025-08-29 04:57:45.517113: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 96)       │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 8, 8, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 128)      │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,922 (823.91 KB)

 Trainable params: 210,922 (823.91 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
optimizer = tf.keras.optimizers.AdamW(
    learning_rate=0.001,
    weight_decay=1e-4
)

In [7]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [8]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=25,
    restore_best_weights=True
)

In [9]:
history = model.fit(
    x_train,y_train,
    validation_data=(x_test, y_test),
    epochs=200,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 41ms/step - accuracy: 0.2241 - loss: 2.0331 - val_accuracy: 0.4783 - val_loss: 1.4053
Epoch 2/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 0.4744 - loss: 1.4256 - val_accuracy: 0.5951 - val_loss: 1.1246
Epoch 3/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 0.5899 - loss: 1.1676 - val_accuracy: 0.6586 - val_loss: 0.9642
Epoch 4/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 0.6539 - loss: 1.0087 - val_accuracy: 0.6834 - val_loss: 0.9189
Epoch 5/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 63s 40ms/step - accuracy: 0.6943 - loss: 0.9112 - val_accuracy: 0.6859 - val_loss: 0.9145
Epoch 6/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 0.7230 - loss: 0.8229 - val_accuracy: 0.7130 - val_loss: 0.8600
Epoch 7/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 63s 40ms/step - accuracy: 0.7493 - loss: 0.7640 - val_accuracy: 0.7262 - val_loss: 0.8395
Epoch 8/200
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 